In [1]:
import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

from keras.models import Model
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, Lambda
from keras.applications import inception_resnet_v2
# from keras.preprocessing.image import *

import numpy as np
from tqdm import tqdm
import cv2

np.random.seed(2017)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
n = 25000
img_height, img_width = 299, 299
X = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
y = np.zeros((n, 1), dtype=np.uint8)

for i in tqdm(range(12500)): # n/2
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' % i), (img_height, img_width))
    X[i+12500] = cv2.resize(cv2.imread('train/dog.%d.jpg' % i), (img_height, img_width)) # n/2

y[12500:] = 1

100%|██████████| 12500/12500 [00:55<00:00, 224.64it/s]


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [4]:
i_input_tensor = Input((img_height, img_width, 3))
# x_input_tensor = inception_resnet_v2.preprocess_input(i_input_tensor)	# 错误的处理
x_input_tensor = Lambda(inception_resnet_v2.preprocess_input)(i_input_tensor)

base_model = inception_resnet_v2.InceptionResNetV2(input_tensor=x_input_tensor, weights='imagenet', include_top=False)

In [7]:
for layers in base_model.layers:
    layers.trainable = False

i_output = GlobalAveragePooling2D()(base_model.output)
i_output = Dropout(0.25)(i_output) # 0.25 比 0.5 好
i_predictions = Dense(1, activation='sigmoid')(i_output)
model = Model(base_model.input, i_predictions)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, batch_size=16, epochs=5, validation_data=(X_valid, y_valid))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 185s 9ms/step - loss: 0.1456 - acc: 0.9495 - val_loss: 0.0638 - val_acc: 0.9812
Epoch 2/5
20000/20000 [==============================] - 218s 11ms/step - loss: 0.0953 - acc: 0.9645 - val_loss: 0.0674 - val_acc: 0.9790
Epoch 3/5
20000/20000 [==============================] - 222s 11ms/step - loss: 0.0945 - acc: 0.9660 - val_loss: 0.0477 - val_acc: 0.9862
Epoch 4/5
20000/20000 [==============================] - 219s 11ms/step - loss: 0.0874 - acc: 0.9660 - val_loss: 0.0767 - val_acc: 0.9768
Epoch 5/5
20000/20000 [==============================] - 219s 11ms/step - loss: 0.0907 - acc: 0.9661 - val_loss: 0.0837 - val_acc: 0.9752


In [16]:
for i in range(len(model.layers)):
    print(model.layers[i].name, i)

input_1 0
lambda_1 1
conv2d_1 2
batch_normalization_1 3
activation_1 4
conv2d_2 5
batch_normalization_2 6
activation_2 7
conv2d_3 8
batch_normalization_3 9
activation_3 10
max_pooling2d_1 11
conv2d_4 12
batch_normalization_4 13
activation_4 14
conv2d_5 15
batch_normalization_5 16
activation_5 17
max_pooling2d_2 18
conv2d_9 19
batch_normalization_9 20
activation_9 21
conv2d_7 22
conv2d_10 23
batch_normalization_7 24
batch_normalization_10 25
activation_7 26
activation_10 27
average_pooling2d_1 28
conv2d_6 29
conv2d_8 30
conv2d_11 31
conv2d_12 32
batch_normalization_6 33
batch_normalization_8 34
batch_normalization_11 35
batch_normalization_12 36
activation_6 37
activation_8 38
activation_11 39
activation_12 40
mixed_5b 41
conv2d_16 42
batch_normalization_16 43
activation_16 44
conv2d_14 45
conv2d_17 46
batch_normalization_14 47
batch_normalization_17 48
activation_14 49
activation_17 50
conv2d_13 51
conv2d_15 52
conv2d_18 53
batch_normalization_13 54
batch_normalization_15 55
batch_norm

In [15]:
base_weights = model.layers[783].get_weights()[0]

In [ ]:
model_2 = Model(model.input, [model])

In [17]:
for layer in model.layers[760:]:
    layer.trainable = True

In [20]:
model.fit(X_train, y_train, batch_size=16, epochs=5, validation_data=(X_valid, y_valid))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
   32/20000 [..............................] - ETA: 1:57 - loss: 0.0377 - acc: 1.0000

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


20000/20000 [==============================] - 142s 7ms/step - loss: 0.0901 - acc: 0.9669 - val_loss: 0.0523 - val_acc: 0.9850
Epoch 2/5
20000/20000 [==============================] - 190s 9ms/step - loss: 0.0879 - acc: 0.9669 - val_loss: 0.0693 - val_acc: 0.9806
Epoch 3/5
20000/20000 [==============================] - 214s 11ms/step - loss: 0.0853 - acc: 0.9671 - val_loss: 0.0453 - val_acc: 0.9868
Epoch 4/5
20000/20000 [==============================] - 218s 11ms/step - loss: 0.0828 - acc: 0.9697 - val_loss: 0.0653 - val_acc: 0.9824
Epoch 5/5
20000/20000 [==============================] - 227s 11ms/step - loss: 0.0816 - acc: 0.9700 - val_loss: 0.0774 - val_acc: 0.9786


In [21]:
model.save('modelInceptionResNet_transfer_0_07.h5')

In [26]:
X_test = np.zeros((n, img_height, img_width, 3), dtype=np.uint8)
for i in tqdm(range(12500)):
    j = i+1
    X_test[i] = cv2.resize(cv2.imread('test/%d.jpg' % j), (img_height, img_width))
#     X_test[i] = cv2.resize(cv2.imread('test/%d.jpg' % j, (img_height, img_width))) the wrong code


100%|██████████| 12500/12500 [00:27<00:00, 458.28it/s]


In [27]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

25000/25000 [==============================] - 148s 6ms/step


In [28]:
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

In [30]:
import os
rmrf_mkdir('testRun')
os.symlink('../test/', 'testRun/test')

In [32]:
import pandas as pd
from keras.preprocessing import image

df = pd.read_csv("sample_submission.csv")

image_size = (224, 224)
gen = image.ImageDataGenerator()
test_generator = gen.flow_from_directory("testRun", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('pred.csv', index=None)
df.head(10)

Found 12500 images belonging to 1 classes.


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


id     label
0   1  0.995000
1   2  0.995000
2   3  0.995000
3   4  0.005000
4   5  0.005000
5   6  0.398393
6   7  0.965138
7   8  0.995000
8   9  0.005000
9  10  0.991249